Rahmanda Afebrio Yuris Soesatyo - Chapter 13:,Loading and Preprocessing Data with TensorFlow

1. Dasar-dasar Data API

tf.data API menyediakan abstraksi Dataset, yaitu representasi urutan data (sequence of elements) yang bisa berasal dari memori maupun file. Dataset ini dirancang supaya proses loading dan preprocessing data bisa dilakukan cepat, efisien, dan scalable.

Salah satu konsep penting di tf.data adalah immutability. Artinya, setiap operasi seperti map, batch, atau repeat tidak mengubah Dataset yang ada, melainkan menghasilkan Dataset baru. Karena itu, transformasi data bisa dirangkai dengan mudah menggunakan method chaining, membentuk pipeline yang fleksibel dan mudah dibaca.

Operasi Inti pada Dataset

Beberapa operasi utama yang sering digunakan dalam tf.data.Dataset antara lain:

map
Menerapkan fungsi preprocessing ke setiap elemen dataset.

filter
Menyaring data berdasarkan kondisi tertentu.

batch / unbatch
Mengelompokkan data ke dalam batch atau memecah batch kembali menjadi elemen tunggal.

repeat
Mengulang dataset untuk beberapa epoch.

shuffle
Mengacak urutan data agar distribusi lebih acak.

take
Mengambil sejumlah elemen pertama dari dataset.

interleave
Membaca data dari beberapa sumber secara paralel untuk meningkatkan throughput.

prefetch
Melakukan overlap antara preprocessing di CPU dan training di GPU, sehingga pipeline tidak menjadi bottleneck.

Operasi-operasi ini umumnya dikombinasikan untuk membangun pipeline training yang i.i.d., teracak

In [1]:
import tensorflow as tf

# 1) Dataset sederhana dari tensor
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)

for item in dataset:
    print(item)  # 0..9

# 2) Chaining: repeat, batch
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

# 3) Map: preprocessing per item
dataset = dataset.map(lambda x: x * 2)  # 0,2,4,...

# 4) Unbatch (experimental) + filter + take
dataset = dataset.apply(tf.data.experimental.unbatch())
dataset = dataset.filter(lambda x: x < 10)
for item in dataset.take(3):
    print(item)

# 5) Shuffle + batch
dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)
for batch in dataset:
    print(batch)

Instructions for updating:
Use `tf.data.Dataset.unbatch()`.


tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)
tf.Tensor([3 6], shape=(2,), dtype=int64)


2. CSV Input Pipelines dengan tf.data

Untuk dataset tabular berukuran besar—seperti California Housing—dibutuhkan pipeline input yang efisien agar proses training tidak terhambat oleh I/O. Pola pipeline yang umum digunakan biasanya sebagai berikut:

list_files
Mengumpulkan daftar file CSV yang akan dibaca.

interleave dengan beberapa TextLineDataset
Membaca banyak file secara paralel, sambil melewati baris header.

map(preprocess)
Menerapkan fungsi preprocessing ke setiap baris data.

shuffle
Mengacak data untuk menghindari bias urutan.

repeat (opsional)
Mengulang dataset untuk beberapa epoch.

batch
Mengelompokkan data ke dalam batch.

prefetch
Menjalankan preprocessing di CPU secara paralel dengan training di GPU.

Pipeline seperti ini memungkinkan:

pembacaan data dari banyak file secara paralel,

proses shuffling yang lebih efektif,

overlap antara loading/preprocessing dan proses training,

sehingga bottleneck I/O bisa diminimalkan secara signifikan.

Preprocessing Data CSV

Tahap preprocessing CSV biasanya dilakukan langsung di dalam pipeline tf.data, dengan memanfaatkan:

tf.io.decode_csv
Untuk mem-parsing setiap baris CSV menjadi tensor fitur dan label.

Operasi scaling
Seperti normalisasi atau standardisasi, yang diterapkan langsung setelah parsing.

Dengan pendekatan ini, seluruh proses input data—mulai dari membaca file hingga preprocessing—tetap berada di dalam TensorFlow computation graph, sehingga bisa dioptimalkan secara end-to-end dan berjalan lebih efisien.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Precomputed mean & std per feature (misal dari NumPy)
X_mean = tf.constant([...], dtype=tf.float32)
X_std  = tf.constant([...], dtype=tf.float32)
n_inputs = 8

def preprocess(line):
    # record_defaults: 8 feature float (default 0.), 1 target float (no default)
    defs = [0.] * n_inputs + [tf.constant((), dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    x = (x - X_mean) / X_std
    return x, y

def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths)
    dataset = dataset.interleave(
        lambda fp: tf.data.TextLineDataset(fp).skip(1),
        cycle_length=n_readers,
        num_parallel_calls=n_read_threads
    )
    dataset = dataset.map(preprocess,
                          num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size).repeat(repeat)
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

train_set = csv_reader_dataset(train_filepaths)
valid_set = csv_reader_dataset(valid_filepaths)
test_set  = csv_reader_dataset(test_filepaths)

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu",
                       kernel_initializer="he_normal"),
    keras.layers.Dense(1)
])
model.compile(loss="mse", optimizer="nadam")
model.fit(train_set, epochs=10, validation_data=valid_set)
model.evaluate(test_set)


3. TFRecord Format & Protocol Buffers

TFRecord merupakan format file biner sederhana yang menyimpan data sebagai rangkaian record dengan panjang variabel. Format ini sangat cocok digunakan untuk dataset berskala besar dan bertipe kompleks—seperti gambar, audio, maupun tensor arbitrer—karena:

proses baca jauh lebih cepat dibandingkan format teks (misalnya CSV),

ukuran file lebih ringkas dan efisien,

mudah diparalelkan dalam pipeline input TensorFlow.

Dalam praktiknya, setiap record TFRecord biasanya berisi objek Protocol Buffers khusus TensorFlow, yaitu:

Example
Digunakan untuk satu data contoh dengan struktur tetap (fixed-length).

SequenceExample
Digunakan untuk data berurutan, seperti time series, teks, atau urutan frame.

Proses parsing dilakukan sepenuhnya di dalam TensorFlow menggunakan operasi bawaan, seperti:

tf.io.parse_single_example

tf.io.parse_single_sequence_example

Typical TFRecord Pipeline

Penggunaan TFRecord umumnya dibagi menjadi dua tahap utama:

Konversi Offline
Data mentah (misalnya CSV atau folder gambar) diubah terlebih dahulu menjadi file TFRecord yang berisi objek Example atau SequenceExample.

Pipeline Saat Training
Pada tahap pelatihan, pipeline biasanya meliputi:

TFRecordDataset untuk membaca file,

parsing protobuf menjadi tensor,

preprocessing standar seperti map, batch, dan prefetch.

Pendekatan ini memisahkan proses preprocessing berat dari training loop, sehingga kinerja I/O meningkat secara signifikan dan proses pelatihan menjadi lebih efisien serta stabil.

In [4]:
import tensorflow as tf

# Write raw strings to TFRecord
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"This is the first record")
    f.write(b"And this is the second record")

# Read TFRecord
dataset = tf.data.TFRecordDataset(["my_data.tfrecord"])
for item in dataset:
    print(item)  # scalar string tensors


tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'And this is the second record', shape=(), dtype=string)


In [5]:

from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example

# Build Example
person_example = Example(
    features=Features(
        feature={
            "name":   Feature(bytes_list=BytesList(value=[b"Alice"])),
            "id":     Feature(int64_list=Int64List(value=[123])),
            "emails": Feature(bytes_list=BytesList(
                value=[b"a@b.com", b"c@d.com"]
            )),
        }
    )
)

with tf.io.TFRecordWriter("my_contacts.tfrecord") as f:
    f.write(person_example.SerializeToString())

# Parse with tf.io.parse_single_example
feature_description = {
    "name":   tf.io.FixedLenFeature([], tf.string, default_value=""),
    "id":     tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "emails": tf.io.VarLenFeature(tf.string),
}

for serialized in tf.data.TFRecordDataset(["my_contacts.tfrecord"]):
    parsed = tf.io.parse_single_example(serialized, feature_description)
    name   = parsed["name"]
    pid    = parsed["id"]
    emails = parsed["emails"].values  # sparse → dense values


4. Preprocessing Numerical & Categorical Features

Preprocessing fitur numerik dan kategorikal dapat dilakukan pada beberapa level, tergantung kebutuhan eksperimen dan target deployment model.

Level Preprocessing

Di luar TensorFlow
Menggunakan library seperti NumPy, pandas, atau scikit-learn, misalnya dengan:

StandardScaler untuk standardisasi fitur numerik

OneHotEncoder untuk fitur kategorikal

Pendekatan ini cocok untuk eksplorasi awal, tetapi kurang ideal untuk production karena preprocessing terpisah dari model.

Di dalam pipeline tf.data
Preprocessing dilakukan melalui map(preprocess_fn) sehingga:

berjalan paralel,

terintegrasi langsung dengan pipeline training,

mengurangi overhead pemrosesan terpisah.

Sebagai preprocessing layer di dalam model Keras
Pendekatan ini paling direkomendasikan untuk production, karena:

preprocessing ikut tersimpan dan diekspor bersama model,

konsistensi antara training dan inference terjamin,

mengurangi risiko perbedaan logika preprocessing di lingkungan berbeda.

Contoh Pendekatan

Standardization layer
Layer kustom atau bawaan Keras yang menggunakan metode adapt, dengan konsep serupa StandardScaler.

Encoding fitur kategorikal, menggunakan:

lookup table (StringLookup atau IntegerLookup),

one-hot encoding,

atau embedding layer untuk fitur kategorikal dengan kardinalitas besar.

Pendekatan berbasis layer preprocessing memastikan bahwa alur data tetap konsisten dari tahap pelatihan hingga inferensi, sehingga lebih andal untuk sistem machine learning di lingkungan produksi.

In [ ]:
import numpy as np
from tensorflow import keras

class Standardization(keras.layers.Layer):
    def adapt(self, data_sample):
        self.means_ = np.mean(data_sample, axis=0, keepdims=True)
        self.stds_  = np.std(data_sample, axis=0, keepdims=True)

    def call(self, inputs):
        eps = keras.backend.epsilon()
        return (inputs - self.means_) / (self.stds_ + eps)

# Usage
std_layer = Standardization()
std_layer.adapt(X_train_sample)

model = keras.models.Sequential([
    std_layer,
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

In [ ]:
import tensorflow as tf

vocab = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]
indices = tf.range(len(vocab), dtype=tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)

num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

categories = tf.constant(["NEAR BAY", "DESERT", "INLAND", "INLAND"])
cat_indices = table.lookup(categories)

cat_one_hot = tf.one_hot(
    cat_indices,
    depth=len(vocab) + num_oov_buckets
)

5. TF Transform & Keras Preprocessing Layers
TensorFlow Transform (TF Transform / TFT) adalah bagian dari TFX yang memungkinkan preprocessing didefinisikan satu kali dan digunakan secara konsisten untuk training dan serving.

TF Transform bekerja dengan cara:

menjalankan preprocessing secara batch pada seluruh training set menggunakan Apache Beam,
lalu mengompilasi preprocessing tersebut menjadi TF Function yang dapat ditanam langsung ke model saat serving.
Keunggulan utama pendekatan ini adalah menghindari train/serving skew, karena preprocessing yang sama persis digunakan di kedua fase.

Keras Preprocessing Layers
Sebagai alternatif yang lebih ringan, Keras menyediakan preprocessing layers bawaan, antara lain:

Normalization
Discretization
TextVectorization
StringLookup / IntegerLookup
Pola penggunaannya umumnya:

Membuat layer preprocessing
Memanggil adapt(data_sample)
Menggunakannya sebagai layer pertama dalam model
Pendekatan ini mudah diintegrasikan dan cocok untuk sebagian besar use case non-TFX.

In [ ]:
import tensorflow_transform as tft

def preprocess(inputs):
    median_age      = inputs["housing_median_age"]
    ocean_proximity = inputs["ocean_proximity"]

    standardized_age = tft.scale_to_z_score(median_age)
    ocean_id = tft.compute_and_apply_vocabulary(ocean_proximity)

    return {
        "standardized_median_age": standardized_age,
        "ocean_proximity_id": ocean_id,
    }

6. TensorFlow Datasets (TFDS)
TensorFlow Datasets (TFDS) menyediakan akses mudah ke berbagai dataset populer, seperti:

MNIST, Fashion-MNIST
CIFAR
ImageNet
berbagai dataset NLP dan speech
TFDS secara otomatis:

mendownload dataset,
memverifikasi checksum,
dan menyajikannya sebagai tf.data.Dataset siap pakai.
TFDS Usage Patterns
tfds.load() dapat mengembalikan:

dictionary dataset (misalnya {"train": ds_train, "test": ds_test}), atau
pasangan (features, labels) jika as_supervised=True.
Pipeline yang umum digunakan:

tfds.load
shuffle
batch
prefetch
langsung digunakan pada model.fit()
TFDS sangat membantu untuk eksperimen cepat, benchmarking, dan pembelajaran tanpa harus menulis pipeline input dari nol.

In [ ]:
import tensorflow_datasets as tfds
from tensorflow import keras

dataset = tfds.load(
    name="mnist",
    batch_size=32,
    as_supervised=True
)
mnist_train, mnist_test = dataset["train"], dataset["test"]

mnist_train = mnist_train.prefetch(1)

model = keras.models.Sequential([
    keras.layers.Reshape([28, 28, 1], input_shape=[28, 28, 1]),
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax"),
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

model.fit(mnist_train, epochs=5)
